In [1]:
#test_ignore
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tsai.all import *
import zarr

In [3]:
def assign_in_chunks_with_concat(a, b, concat_start = 0, chunksize='auto', inplace=True, verbose=True):
    """Assigns values in b to an array-like object a using chunks to avoid memory overload.

    The resulting a retains it's dtype and share it's memory.
    a: array-like object
    b: may be an integer, float, str, 'rand' (for random data), or another array like object.
    chunksize: is the size of chunks. If 'auto' chunks will have around 1GB each.
    """

    if b != 'rand' and not isinstance(b, (Iterable, Generator)):
        a[:] = b
    else:
        shape = b.shape
        dtype = a.dtype
        if chunksize == "auto":
            chunksize = chunks_calculator(shape, dtype)
            chunksize = shape[0] if not chunksize else  chunksize[0]
        for i in progress_bar(range((shape[0] - 1) // chunksize + 1), display=verbose, leave=False):
            start, end = i * chunksize, min(shape[0], (i + 1) * chunksize)
            if start >= shape[0]: break
            if b == 'rand':
                a[start:end] = np.random.rand(end - start, *shape[1:])
            else:
                a[start + concat_start:end + concat_start] = b[start:end]
    if not inplace: return a

In [5]:
file_names = ['/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_0_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_1_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_2_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_3_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_4_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_5_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_6_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_7_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_8_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_9_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtest_batch_0_ca_co_day1above_3h_on_disk.npy',
 '/media/scottcha/Data2/OAPMLData//5.MLData/Xtest_batch_1_ca_co_day1above_3h_on_disk.npy']

In [9]:
zarr_arr = zarr.open('/media/scottcha/Data2/OAPMLData/5.MLData/tmp.zarr', mode='w', chunks=(1,-1, -1), shape=(12*2500, 774, 1441), dtype=np.float32)

In [6]:
file_size = 2500
for fi in range(len(file_names)):
    print('On ' + str(fi))
    tmpnp = np.load(file_names[fi], mmap_mode='r')
    assign_in_chunks_with_concat(zarr_arr, tmpnp, concat_start=file_size * fi)

In [10]:
tmpnp2 = np.load('/media/scottcha/Data2/OAPMLData//5.MLData/Xtrain_batch_1_ca_co_day1above_3h_on_disk.npy', mmap_mode='r')

In [11]:
zarr_arr.shape

(5000, 774, 1441)

In [20]:
zarr_arr[1]

array([[     nan,      nan,      nan, ..., 8.20e-05, 1.26e-04, 1.08e-04],
       [     nan,      nan,      nan, ..., 8.20e-05, 1.26e-04, 1.08e-04],
       [     nan,      nan,      nan, ..., 8.20e-05, 1.26e-04, 1.08e-04],
       ...,
       [     nan,      nan,      nan, ..., 3.50e+01, 4.10e+01, 4.20e+01],
       [     nan,      nan,      nan, ..., 3.50e+01, 4.10e+01, 4.20e+01],
       [     nan,      nan,      nan, ..., 3.50e+01, 4.10e+01, 4.20e+01]],
      dtype=float32)

In [17]:
assign_in_chunks_with_concat(zarr_arr, tmpnp2, concat_start=2500)